In [1]:
import numpy
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import ipykernel
# import gym

In [2]:
print(numpy.version.version)
print(tf.config.list_physical_devices())
print(ipykernel.__version__)


1.23.3
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
6.15.2


In [6]:
highscore = {"commands":[{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0}],"score":421}

In [7]:
commands = []
for kv in highscore["commands"]:
  power = kv['power']
  rotation = kv['rotation']
  commands.append(f"{rotation} {power}")

commands[:5]

['-43 4', '-43 4', '-43 4', '-43 4', '-43 4']

In [8]:
import subprocess
from collections import deque

replay_commands = deque(commands)

node_path = "/usr/local/bin/node"
script_path = "../cli/target/scala-2.13/cli-fastopt/main.js"

p = subprocess.Popen([node_path, script_path], bufsize=1, stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, text=True)
level_name = p.stdout.readline()
print(f"Level: {level_name}")
num_landing_coords = int(p.stdout.readline())
landing_coords = []
states = []
for _ in range(num_landing_coords):
  landing_coords.append(p.stdout.readline())

initial_state = p.stdout.readline()
states.append(initial_state)

print(f"num_landing_coords: {num_landing_coords}")
print(f"landing_coords: {landing_coords}")
print(initial_state)

poll = p.poll()
while poll is None and len(replay_commands) > 0:
  print("sending command")
  command = replay_commands.popleft()
  p.stdin.writelines([f"{command}\n"])
  print(f'command "{command}" sent')
  
  new_state = p.stdout.readline()
  states.append(new_state)
  # new_state = p.stdout.readline()
  # print(new_state)  
  poll = p.poll()
  print(f"poll: {poll}")



Level: Level: 2-3

num_landing_coords: 7
landing_coords: ['0 100\n', '1000 500\n', '1500 1500\n', '3000 1000\n', '4000 150\n', '5500 150\n', '6999 800\n']
{"distanceLandingArea":2832.4018076537095,"fuel":750,"hSpeed":-90,"horizontalDistanceLandingArea":-1000,"isCrashed":false,"isLanded":false,"isOffLimits":false,"isOutOfFuel":false,"landing-distance-0":3643.1435601690196,"landing-distance-100":2832.4021607111163,"landing-distance-25":3396.78125215626,"landing-distance-50":3175.6889016400837,"landing-distance-75":2985.4859529062014,"landing-path-free-0":true,"landing-path-free-100":true,"landing-path-free-25":true,"landing-path-free-50":true,"landing-path-free-75":true,"power":0,"radar-maybe-intersection-distance--10":null,"radar-maybe-intersection-distance--15":null,"radar-maybe-intersection-distance--20":null,"radar-maybe-intersection-distance--25":null,"radar-maybe-intersection-distance--30":null,"radar-maybe-intersection-distance--35":null,"radar-maybe-intersection-distance--40":nul

In [9]:
len(states)

90

In [10]:
type(states[-1])

str

In [11]:
ary = []
for s in states:
  ary.append(json.loads(s))
 
json_ary = json.dumps(ary)

In [12]:
df = pd.read_json(json_ary)
df


,distanceLandingArea,fuel,hSpeed,horizontalDistanceLandingArea,isCrashed,isLanded,isOffLimits,isOutOfFuel,landing-distance-0,landing-distance-100,...,radar-maybe-intersection-distance-75,radar-maybe-intersection-distance-80,radar-maybe-intersection-distance-85,radar-maybe-intersection-distance-90,radar-maybe-intersection-distance-95,rotation,vSpeed,verticalDistanceLandingArea,x,y
0,2832.401808,750,-90.000000,-1000.000000,False,False,False,False,3643.143560,2832.402161,...,NaN,NaN,NaN,NaN,NaN,90,0.000000,2650.000000,6500.000000,2800.000000
1,2800.102845,749,-90.965926,-909.517037,False,False,False,False,3580.380180,2800.103169,...,NaN,NaN,NaN,NaN,NaN,75,-3.452181,2648.273910,6409.517037,2798.273910
2,2767.042212,747,-92.697977,-817.685086,False,False,False,False,3515.618470,2767.042507,...,NaN,NaN,NaN,NaN,NaN,60,-6.163181,2643.466229,6317.685086,2793.466229
3,2734.089434,744,-94.819297,-723.926449,False,False,False,False,3449.205696,2734.089699,...,NaN,NaN,NaN,NaN,NaN,45,-7.752861,2636.508208,6223.926449,2786.508208
4,2702.632837,740,-96.819297,-628.107152,False,False,False,False,3382.091283,2702.633069,...,NaN,NaN,NaN,NaN,NaN,30,-7.999759,2628.631898,6128.107152,2778.631898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,187.381207,437,1.400397,0.000000,False,False,False,False,188.294483,1493.278388,...,361.295001,306.172380,267.408952,238.972302,217.498495,0,-39.515081,187.381207,4018.523848,337.381207
86,148.010626,433,1.400397,0.000000,False,False,False,False,149.345509,1487.457034,...,293.384941,248.623328,217.145987,194.054372,176.616845,0,-39.226081,148.010626,4019.924244,298.010626
87,108.929045,429,1.400397,0.000000,False,False,False,False,110.996551,1482.681151,...,225.988917,191.509886,167.263481,149.476443,136.044643,0,-38.937081,108.929045,4021.324641,258.929045
88,70.136464,425,1.400397,0.000000,False,False,False,False,73.725880,1478.937958,...,159.106928,134.832053,117.761433,105.238513,95.781888,0,-38.648081,70.136464,4022.725038,220.136464
